In [ ]:
!pip install --quiet duckdb
# !pip install --quiet jupysql 
# !pip install --quiet duckdb-engine
!pip install --quiet chart-studiot
!pip install --quiet pycaret 
!pip install --quiet pandas_profiling
!pip install --quiet dython
!pip install --quiet numpy  --upgrade --user
!pip install --quiet dowhy  --upgrade --user

# Objective:

### 1. Analyze driver’s Lifetime Value (LTV) 
### 2. Using the data provided, identify trends and insights of the factors that impact LTV

In [ ]:
import warnings
warnings.simplefilter("ignore")

# data engineering
import duckdb
# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# data processing, compute, auto-EDA
import numpy as np
import scipy
from scipy import stats
import pandas as pd 
from pandas_profiling import ProfileReport

# visualizations for EDA
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# correlation scores
# from sklearn.metrics import jaccard_score
# from dython.nominal import associations
# from dython.nominal import identify_nominal_columns

# feature importance
# from pycaret.classification import *
# from pycaret.regression import *

# causality inference
# from dowhy import CausalModel

# Data

In [ ]:
drivers = pd.read_csv('/kaggle/input/lyftdatachallenge/driver_ids.csv')
rides = pd.read_csv('/kaggle/input/lyftdatachallenge/ride_ids.csv')
ride_timestamps = pd.read_csv('/kaggle/input/lyftdatachallenge/ride_timestamps.csv')
# combined = pd.read_csv('/kaggle/input/lyft-analysis/combined_lyft_dataset.csv')

In [ ]:
drivers.head()

In [ ]:
rides.head()

In [ ]:
ride_timestamps.shape

In [ ]:
ride_timestamps.head(10)

# Data Quality Check- duplications, outliers, non-sense data

In [ ]:
query = """
select count(*) as total_rows
from drivers
"""

duckdb.query(query).df()

In [ ]:
query = """
select count(distinct driver_id) as unique_drivers
from drivers
"""

duckdb.query(query).df() # same as total rows 

In [ ]:
query = """
select count(*) as total_ride_rows
from rides
"""

duckdb.query(query).df() 

In [ ]:
query = """
select count(distinct ride_id) as unique_rides
from rides
"""

duckdb.query(query).df() # same as total rows

In [ ]:
query = """
select *
from rides
"""

duckdb.query(query).df() 

In [ ]:
query = """
select min(ride_distance)
, max(ride_distance)
, min(ride_duration)
, max(ride_duration)
from rides
"""

duckdb.query(query).df()

In [ ]:
rides.boxplot(column='ride_distance', return_type='axes')

In [ ]:
rides.ride_distance.describe()

In [ ]:
duckdb.query('select ride_distance from rides where ride_distance > 200000').df()

In [ ]:
duckdb.query('select ride_distance, ride_duration from rides where ride_distance > 200000').df()

In [ ]:
clean_rides = duckdb.query('select * from rides where ride_distance between 0 and 200000').df()
clean_rides.boxplot(column='ride_distance', return_type='axes')

In [ ]:
clean_rides.ride_distance.describe()

In [ ]:
clean_rides.boxplot(column='ride_duration', return_type='axes')

In [ ]:
clean_rides.ride_duration.describe()

In [ ]:
clean_rides.head()

In [ ]:
# Take out anything beyond 10,000 seconds/2.8 hours and do some conversions

query = """
select driver_id
, ride_id
, ride_distance * 0.000621371192 as ride_distance
, ride_duration/60 as ride_duration
, ride_prime_time/100 as ride_prime_time
from clean_rides
where ride_duration < 10000
"""


clean_rides2 = duckdb.query(query).df()
clean_rides2.boxplot(column='ride_duration', return_type='axes')

In [ ]:
# Seeing if z-score creates different distribution
clean_rides3 = clean_rides[(np.abs(stats.zscore(clean_rides.drop(['driver_id', 'ride_id', 'ride_distance', 'ride_prime_time'], 
                                                       axis=1))) < 3).all(axis=1)]
clean_rides3.boxplot(column='ride_duration', return_type='axes')

Extreme filtering with z-score. 
Final dataset will have abnormal, long tail data with distance, duration, which depicts real driver life.

In [ ]:
clean_rides2.ride_prime_time.value_counts()

In [ ]:
# Checking that each distinct ride has all 5 events

query = """
with ride_event_freq as (
select ride_id
, count(event) as event_num
from ride_timestamps
group by 1
)

select event_num
, count(*) as event_num_freq
from ride_event_freq
group by 1
order by 2
"""

duckdb.query(query).df() # event_num_freq same as total unique rides

In [ ]:
# Checking that each subsequent event has later timestamp
query = """
with time_test as (
select *
, lead(event, 1) over (partition by ride_id order by timestamp) as subsequent_event
from ride_timestamps
)

select count(*)
from time_test
where subsequent_event IS NULL
"""

duckdb.query(query).df() 

Great! This is expected since for each unique ride, suppose to have last event (dropped_off_at as NULL). And got count that matches unique rides.

Final check is to see if ride timestamps are greater than when drivers were onboarded.

In [ ]:
drivers['driver_onboard_date'] = pd.to_datetime(drivers['driver_onboard_date'])
ride_timestamps['timestamp'] = pd.to_datetime(ride_timestamps['timestamp'])

In [ ]:
drivers.driver_onboard_date.min(), drivers.driver_onboard_date.max()

In [ ]:
ride_timestamps.timestamp.min(), ride_timestamps.timestamp.max()

Yes, ride timestamps are later than driver onboard dates.

In [ ]:
type(drivers['driver_onboard_date'][0])

In [ ]:
type(ride_timestamps['timestamp'][0])

In [ ]:
# Using SQL to determine duration to subsequent step in rider journey
query = """
with time_test as (
select *
, lead(timestamp, 1) over (partition by ride_id order by timestamp) as next_event_time
from ride_timestamps
)

select *
, case when event = 'requested_at' then date_diff('second', timestamp, next_event_time) end as accept_dur_sec
, case when event = 'accepted_at' then date_diff('second', timestamp, next_event_time)/60 end as arrive_dur_min
, case when event = 'arrived_at' then date_diff('second', timestamp, next_event_time) end as picked_up_dur_sec
, case when event = 'picked_up_at' then date_diff('second', timestamp, next_event_time)/60 end as dropped_off_dur_min
from time_test
order by ride_id, timestamp
"""

rider_journey = duckdb.query(query).df() 
rider_journey

In [ ]:
ride_dur = rider_journey[['accept_dur_sec',	'arrive_dur_min', 'picked_up_dur_sec', 'dropped_off_dur_min']]

sns.boxplot(data=ride_dur)

Most variance from request -> accept and arrive -> picked up.

In [ ]:
ride_timestamps.head()

In [ ]:
# Pivot rider_timestamp event table so each row is a distinct ride_id
ride_ts_pivot = duckdb.query('PIVOT ride_timestamps ON event USING FIRST(timestamp)').df() 
ride_ts_pivot = ride_ts_pivot[['ride_id', 'requested_at', 'accepted_at', 'arrived_at', 'picked_up_at', 'dropped_off_at']]
ride_ts_pivot = duckdb.query('select * from ride_ts_pivot order by ride_id').df()
ride_ts_pivot.head()

In [ ]:
#X-check
duckdb.query('select * from ride_timestamps order by 1,3 limit 20').df() 

In [ ]:
query = """
select count(distinct drivers.driver_id)/(select count(*) from drivers) as drivers_without_rides_perc
from drivers
left join clean_rides2
on drivers.driver_id = clean_rides2.driver_id
where clean_rides2.driver_id is null
"""

driver_rides = duckdb.query(query).df() 
driver_rides

Reasons for this mismatch:

* not all drivers have started completing rides
* not all rides have drivers from driver onboard time range
* not all rides have ride events logged

In [ ]:
clean_rides2.shape, ride_ts_pivot.shape #more rides in ride events table

In [ ]:
query = """
select count(*)/(select count(*) from ride_ts_pivot) as rides_with_events_without_drivers_perc
from clean_rides2
full join ride_ts_pivot
on clean_rides2.ride_id = ride_ts_pivot.ride_id
where ride_ts_pivot.ride_id IS NULL
"""

ride_events = duckdb.query(query).df() 
ride_events

Will perform INNER join between drivers-rides-ride_events.

Questions that dataset can answer:

1. When drivers approved to drive (if power users tied to time then maybe there was a promo offered during that time to receive bonus with X amount rides completed 3 months out)
2. How many rides done per user
3. What sort of trip lengths accepted by certain users?
4. How long is the usual ride broken down by different user?
5. What is the usual multiplier applied to rides attached to different driver cohorts?
6. What is the usual time between requested_at and dropped_off_at as well as all the usual times between each ride timestamp (requested to accepted, accepted to arrived, arrive to pick up and pick up to drop off)


Extra data that would be nice to help define drivers' LTV:

1. CAC - cost per acquired driver
1. how is revenue distributed in city (cost areas)

# Data Engineering

In [ ]:
query = """
with driver_rides_events as (
SELECT drivers.driver_id
, drivers.driver_onboard_date
, clean_rides2.ride_id
, clean_rides2.ride_distance
, clean_rides2.ride_duration
, clean_rides2.ride_prime_time
, ride_ts_pivot.requested_at	
, ride_ts_pivot.accepted_at	
, ride_ts_pivot.arrived_at	
, ride_ts_pivot.picked_up_at	
, ride_ts_pivot.dropped_off_at
from drivers
join clean_rides2 -- only drivers with rides
on drivers.driver_id = clean_rides2.driver_id
join ride_ts_pivot -- all drivers with rides' events
on clean_rides2.ride_id = ride_ts_pivot.ride_id
)

select *
from driver_rides_events
"""
merged = duckdb.query(query).df()  

## How does the demand for rides impact the number of drivers?

In [ ]:
# Left join all rides to number of drivers, join on driver_id,
# join rides to ride_ts_pivot to get ride requested_at timestamp to have time rides x drivers time series

query = """
select 
extract(week from requested_at) as ride_requested_at_week
, count(clean_rides2.ride_id) as total_rides
, count(distinct drivers.driver_id) as total_drivers
from clean_rides2
left join drivers
on clean_rides2.driver_id = drivers.driver_id
join (select ride_id, requested_at from ride_ts_pivot) as rides
on clean_rides2.ride_id = rides.ride_id
group by 1
order by 1
"""

market = duckdb.query(query).df()

market.plot.line(x='ride_requested_at_week', y= ['total_rides', 'total_drivers'])

In [ ]:
market.plot.line(x='ride_requested_at_week', y='total_rides')

In [ ]:
market.plot.line(x='ride_requested_at_week', y='total_drivers')

After week 19, supply has steeper negative decline than rides. Market imbalance starts here. 

# In order to start examining LTV and Churn, need to do some feature engineering with this dataset at the driver level. 

In [ ]:
merged.head()

In [ ]:
# Add date parts
query = """
select *
, date_part('hour', requested_at) as requested_at_hour
, date_part('day', requested_at) as requested_at_day
, date_part('dayofweek', requested_at) as requested_at_dow
, date_part('week', requested_at) as requested_at_week
, date_part('month', requested_at) as requested_at_month
, date_part('year', requested_at) as requested_at_year

, date_part('hour', accepted_at) as accepted_at_hour
, date_part('day', accepted_at) as accepted_at_day
, date_part('dayofweek', accepted_at) as accepted_at_dow
, date_part('week', accepted_at) as accepted_at_week
, date_part('month', accepted_at) as accepted_at_month
, date_part('year', accepted_at) as accepted_at_year

, date_part('day', arrived_at) as arrived_at_day
, date_part('dayofweek', arrived_at) as arrived_at_dow
, date_part('week', arrived_at) as arrived_at_week
, date_part('month', arrived_at) as arrived_at_month
, date_part('year', arrived_at) as arrived_at_year

, date_part('day', picked_up_at) as picked_up_at_day
, date_part('dayofweek', picked_up_at) as picked_up_at_dow
, date_part('week', picked_up_at) as picked_up_at_week
, date_part('month', picked_up_at) as picked_up_at_month
, date_part('year', picked_up_at) as picked_up_at_year

, date_part('day', dropped_off_at) as dropped_off_at_day
, date_part('dayofweek', dropped_off_at) as dropped_off_at_dow
, date_part('week', dropped_off_at) as dropped_off_at_week
, date_part('month', dropped_off_at) as dropped_off_at_month
, date_part('year', dropped_off_at) as dropped_off_at_year
from merged
"""

merged_datepart = duckdb.query(query).df()
merged_datepart.head(2)

In [ ]:
# Pickup window: time difference between request and accept
query = """
select *
, date_diff('second', requested_at, accepted_at) as accept_dur_sec
, date_diff('minute', picked_up_at, dropped_off_at) as ride_dur_min
from merged_datepart
"""

merged_dur = duckdb.query(query).df()
merged_dur.head(2)

In [ ]:
# Ride Count
query = """
select *
, count(ride_id) over (partition by driver_id) as ride_count
from merged_dur
"""

merged_ride_count = duckdb.query(query).df()
merged_ride_count.head(2)

## Computing fare, we use guidance from Lyft's rate card:

### Fare = (base fare + cost per mile * miles traveled + cost per minute * mins traveled)(1 + prime time/100) + service fee

### = 2 + 1.15 * miles traveled + 0.22 * mins traveled)(1 + prime time/100) + 1.75

### After calculating fare, need to ensure that all fares are with lower bound of 5 USD and upper bound of 400 USD.

In [ ]:
merged_ride_count.head(1)

In [ ]:
# Fare

query = """
with fare_sub as (
select *
, ((2 + 1.15*ride_distance + 0.22*ride_duration)*(1 + ride_prime_time)) + 1.75
as fares
from merged_ride_count
)

select *
, case when fares < 5.0 then 5.0 
when fares > 400.0 then 400.0
else fares
end as fare
from fare_sub
"""

merged_fare = duckdb.query(query).df()
merged_fare.head(2)

In [ ]:
merged_fare.drop(columns=['fares'], inplace=True)

In [ ]:
merged_fare.fare.describe()

In [ ]:
duckdb.query('select driver_id, sum(fare) as total_fare from merged_fare group by 1 order by 2 desc').df()

Total fare is the total income for the driver.

In [ ]:
# Create is_weekday flag to filter out possible part-time weekend drivers

query = """
select *
, case when accepted_at_dow not in (0,6) then 1 else 0 end as is_weekday
from merged_fare
"""

merged_wkday = duckdb.query(query).df()
merged_wkday.head(2)

In [ ]:
merged_wkday.is_weekday.value_counts()

In [ ]:
merged_wkday.accepted_at_hour.value_counts()

In [ ]:
# Flag for drivers who drive late (11 PM PST- 6 AM PST or between 6 and 23; since timestamps in UTC (7 hours ahead of SF (PST)): between 6 and 13

query = """
select *
, case when accepted_at_hour between 6 and 13 then 1 else 0 end as is_late_ride
from merged_wkday
"""

merged_late = duckdb.query(query).df()
merged_late.head(2)

In [ ]:
merged_late.is_late_ride.value_counts(normalize=True)

In [ ]:
#How often a night owl driver?
duckdb.query('select driver_id, avg(is_late_ride) as is_late_ride_avg from merged_late group by 1 order by 2 desc').df()

Looks there are some drivers doing just night shifts.

In [ ]:
# Compute how long it takes for driver to pick up next ride 
# Caluclate time between dropped_off_at from ride 1 to accepted_at from ride 2

# order rides in asc order by driver_id
# condition when dropped_off and accept_at in same month and same day then calculate difference in minutes

query = """
with next_ride as (
select *
, lead(accepted_at, 1) over (partition by driver_id order by accepted_at) as next_ride_acc
from merged_late
)

select *
, case when extract(month from dropped_off_at) = extract(month from next_ride_acc) and extract(day from dropped_off_at) = extract(day from next_ride_acc) then date_diff('minute', dropped_off_at, next_ride_acc) else null end as next_ride_lag
from next_ride
"""

merged_next_ride = duckdb.query(query).df()
duckdb.query('select driver_id, requested_at, accepted_at, dropped_off_at, next_ride_acc, next_ride_lag from merged_next_ride order by driver_id, accepted_at limit 25').df()

In [ ]:
merged_next_ride.next_ride_lag.describe()

In [ ]:
merged_next_ride.drop(columns=['next_ride_acc'], inplace=True)

In [ ]:
# Calculate unique days drivers drove

# get date part from accepted_at
# count distinct accepted_at date partitioned by driver_id for total unique driving days

merged_next_ride['accepted_at_date'] = merged_next_ride['accepted_at'].dt.date
merged_next_ride.head(2)

In [ ]:
merged_next_ride.accepted_at_date.isnull().sum() #dense_rank Ok to use

In [ ]:
query = """
select *
, dense_rank() over (partition by driver_id order by accepted_at_date) 
+ dense_rank() over (partition by driver_id order by accepted_at_date desc) 
- 1 as unique_drive_days
from merged_next_ride
"""

merged_drivedays = duckdb.query(query).df()
duckdb.query('select driver_id, accepted_at_date, unique_drive_days from merged_drivedays order by driver_id, accepted_at_date').df()

In [ ]:
# total driving period (calculated earlier but adding to main table)
query = """
with drive_seq as (
select *
, row_number() over (partition by driver_id order by accepted_at desc) as drive_order
from merged_drivedays
)

select  *
, case when drive_order = 1 then datediff('day', driver_onboard_date,  accepted_at) else NULL end as driving_period
from drive_seq
"""

merged_drive_pd = duckdb.query(query).df()
merged_drive_pd.head(5)

## Explore how drivers churn once they start with Lyft.
1. Are there any predictive indicators for driver churn?
1. % of active drivers over time (in weekly buckets, cohorted by onboard week)

## Defining churn: 
* get last ride timestamp
* see if any driver has not had a ride within 7 days
* if so, these are inactive/churned drivers

In [ ]:
# Last accepted ride from driver
merged_drive_pd.accepted_at.max()

In [ ]:
# Get days since last accepted ride record

query = """

select *
, case when drive_order = 1 then datediff('day', accepted_at, TIMESTAMP '2016-06-26 23:57:49') else NULL end as last_activity
from merged_drive_pd

"""

merged_last_activity_dur = duckdb.query(query).df()
duckdb.query('select driver_id, accepted_at, drive_order, last_activity from merged_last_activity_dur order by last_activity desc limit 20').df()

In [ ]:
merged_last_activity_dur.last_activity.describe()

In [ ]:
# Creating churn flag

query = """

select *
, case when last_activity >7 then 1 else 0 end as churn
from merged_last_activity_dur

"""

merged_churn = duckdb.query(query).df()
merged_churn.churn.value_counts()

305 drivers have churned. 


1. Number of Rides and Active Drivers over time (in weekly buckets)

## Number of Rides and Active Drivers over time (in weekly buckets)

In [ ]:
# get total rides and total ACTIVE drivers split by requested_at (need to compare this to prior rides vs drivers graph that did not define Active drivers yet)

query = """

select extract(week from requested_at) as requested_at_week
, count(ride_id) as total_rides
, count(distinct case when churn = 0 then driver_id end) as total_active_drivers
from merged_churn
group by 1
order by 1
"""

rides_active_drivers = duckdb.query(query).df()
rides_active_drivers.plot.line(x='requested_at_week', y=['total_rides', 'total_active_drivers'])

In [ ]:
rides_active_drivers.plot.line(x='requested_at_week', y='total_rides')

In [ ]:
rides_active_drivers.plot.line(x='requested_at_week', y='total_active_drivers')

In [ ]:
query = """

with driver_churn as (
select driver_id
, max(churn) as churn
from merged_churn
group by 1)

select sum(churn)/count(*) *100 as churn_perc
from driver_churn

"""

duckdb.query(query).df()

There is not much difference visually even with 64% active drivers accounted.

# % of active drivers over time (in weekly buckets, cohorted by onboard week)

In [ ]:
# 1st column: get onboard date - cohort (attained)
# 2nd column: get total attained for each cohort onboard week
# for period 1,2,3, etc. out from onboard date get total active drivers with x week out from onboard date
# get percent by dividing total active drivers from x week out from onboard date/drivers attained

In [ ]:
# Raw Retention
query = """

select extract(week from driver_onboard_date) as cohort
, count(distinct driver_id) as drivers_attained
, count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) then driver_id else null end)/drivers_attained as retained_wk0_from_onboard
, count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) + 1 then driver_id else null end)/drivers_attained as retained_wk1_from_onboard
, count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) + 2 then driver_id else null end)/drivers_attained as retained_wk2_from_onboard
, count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) + 3 then driver_id else null end)/drivers_attained as retained_wk3_from_onboard
, count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) + 4 then driver_id else null end)/drivers_attained as retained_wk4_from_onboard
, count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) + 5 then driver_id else null end)/drivers_attained as retained_wk5_from_onboard
, count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) + 6 then driver_id else null end)/drivers_attained as retained_wk6_from_onboard

from merged_churn
group by 1
order by 1

"""

retention = duckdb.query(query).df()
retention

In [ ]:
# Retention %'s by attained week with 1 week periods
query = """

select extract(week from driver_onboard_date) as cohort
, count(distinct driver_id) as drivers_attained
, concat(round(count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) then driver_id else null end)/drivers_attained * 100), '%') as retained_wk0_from_onboard_perc
, concat(round(count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) + 1 then driver_id else null end)/drivers_attained * 100), '%') as retained_wk1_from_onboard_perc
, concat(round(count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) + 2 then driver_id else null end)/drivers_attained * 100), '%') as retained_wk2_from_onboard_perc
, concat(round(count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) + 3 then driver_id else null end)/drivers_attained * 100), '%') as retained_wk3_from_onboard_perc
, concat(round(count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) + 4 then driver_id else null end)/drivers_attained * 100), '%') as retained_wk4_from_onboard_perc
, concat(round(count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) + 5 then driver_id else null end)/drivers_attained * 100), '%') as retained_wk5_from_onboard_perc
, concat(round(count(distinct case when churn = 0 and extract(week from accepted_at) = extract(week from driver_onboard_date) + 6 then driver_id else null end)/drivers_attained * 100), '%') as retained_wk6_from_onboard_perc

from merged_churn
group by 1
order by 1

"""

retention2 = duckdb.query(query).df()
retention2

In [ ]:
# Retention WoW %

query = """

select cohort
, drivers_attained
, round((retained_wk1_from_onboard - retained_wk0_from_onboard)/retained_wk0_from_onboard *100, 2) as retained_wk_1_wow_perc
, round((retained_wk2_from_onboard - retained_wk1_from_onboard)/retained_wk1_from_onboard *100, 2) as retained_wk_2_wow_perc
, round((retained_wk3_from_onboard - retained_wk2_from_onboard)/retained_wk2_from_onboard *100, 2) as retained_wk_3_wow_perc
, round((retained_wk4_from_onboard - retained_wk3_from_onboard)/retained_wk3_from_onboard *100, 2) as retained_wk_4_wow_perc
, round((retained_wk5_from_onboard - retained_wk4_from_onboard)/retained_wk4_from_onboard *100, 2) as retained_wk_5_wow_perc
, round((retained_wk6_from_onboard - retained_wk5_from_onboard)/retained_wk5_from_onboard *100, 2) as retained_wk_6_wow_perc

from retention
order by 1
"""

retention3 = duckdb.query(query).df()
retention3

In [ ]:
# Multi-line line graph

query = """

UNPIVOT retention
ON retained_wk0_from_onboard, retained_wk1_from_onboard, retained_wk2_from_onboard, retained_wk3_from_onboard, retained_wk4_from_onboard, retained_wk5_from_onboard, retained_wk6_from_onboard
INTO
    NAME cohort
    VALUE retained;
    
"""
retention_pivot = duckdb.query(query).df()
retention_pivot

In [ ]:
retention_pivot.set_index("cohort:1", inplace=True)
retention_pivot.groupby("cohort")["retained"].plot(legend=True, xlabel="Retention Week", ylabel="Retained", rot=45)

We can look at row-wise (compare a cohort's WoW performance):
1. Some drivers start driving right away (within the same week as they are onboarded - 100% of the drivers from cohort 18 which also has second to smallest cohort)
1. see that some cohorts have slight rise going from left to right (WoW is positive) meaning that some drivers may have started driving late since no condition for eliminating drivers from population who did not start driving at week 1 from onboard date
1. biggest WoW drop off's were 2 weeks out from onboard date
1. after week 2 from onboard date, WoW is not as drastic in change

Looking column-wise (compare different cohorts' retention per week):
1. whatever was done for week cohort 18 was success since all drivers started driving within onboarding week
1. in general, cohort 18 had highest retention rates across weeks until week 4 
out. then retention rates became comparable to other cohorts
1. cohort 16 has relatively high retention rate as well

From the graph:
1. We can see cohort 13 (first one) has the lowest retention. Seems like product/growth team did some work to increase retention rates since succeeding cohorts have higher retention WoW
1. Although cohort 15 and 17 and have the steepest WoW retention following cohort 13, 15 slightly recovers 5 weeks out from onboard date whereas cohort 17 continues to dip.
1. Cohorts 16 & 19 have the best uplift at week 5 from onboard date. 

# Predictive Indicators of Churn and LTV

In [ ]:
# Need to aggregated/fold up table to driver level to look at churn/LTV predictive behaviors

query = """

select 
driver_id,
 avg(ride_distance) AS avg_ride_distance,
 avg(ride_duration) AS avg_ride_duration,
 avg(ride_prime_time) AS avg_ride_prime_time,
 avg(requested_at_hour) AS avg_requested_at_hour,
 avg(requested_at_day) AS avg_requested_at_day,
 avg(requested_at_dow) AS avg_requested_at_dow,
 avg(requested_at_week) AS avg_requested_at_week,
 avg(requested_at_month) AS avg_requested_at_month,
 avg(requested_at_year) AS avg_requested_at_year,
 avg(accepted_at_hour) AS avg_accepted_at_hour,
 avg(accepted_at_day) AS avg_accepted_at_day,
 avg(accepted_at_dow) AS avg_accepted_at_dow,
 avg(accepted_at_week) AS avg_accepted_at_week,
 avg(accepted_at_month) AS avg_accepted_at_month,
 avg(accepted_at_year) AS avg_accepted_at_year,
 avg(arrived_at_day) AS avg_arrived_at_day,
 avg(arrived_at_dow) AS avg_arrived_at_dow,
 avg(arrived_at_week) AS avg_arrived_at_week,
 avg(arrived_at_month) AS avg_arrived_at_month,
 avg(arrived_at_year) AS avg_arrived_at_year,
 avg(picked_up_at_day) AS avg_picked_up_at_day,
 avg(picked_up_at_dow) AS avg_picked_up_at_dow,
 avg(picked_up_at_week) AS avg_picked_up_at_week,
 avg(picked_up_at_month) AS avg_picked_up_at_month,
 avg(picked_up_at_year) AS avg_picked_up_at_year,
 avg(dropped_off_at_day) AS avg_dropped_off_at_day,
 avg(dropped_off_at_dow) AS avg_dropped_off_at_dow,
 avg(dropped_off_at_week) AS avg_dropped_off_at_week,
 avg(dropped_off_at_month) AS avg_dropped_off_at_month,
 avg(dropped_off_at_year) AS avg_dropped_off_at_year,
 avg(accept_dur_sec) AS avg_accept_dur_sec,
 avg(ride_dur_min) AS avg_ride_dur_min,
 max(ride_count) as ride_count,
 avg(fare) as avg_fare,
 mode(is_weekday) as is_weekday,
 mode(is_late_ride) as is_late_ride,
 avg(next_ride_lag) as avg_next_ride_lag,
 max(unique_drive_days) as unique_drive_days,
 max(driving_period) as driving_period,
 max(churn) as churn

from merged_churn
group by ALL

"""

df = duckdb.query(query).df()
df.head()

In [ ]:
#Taking care of NULL values
nullseries = df.isnull().sum()
nullseries[nullseries > 0]

In [ ]:
df.fillna(0, inplace=True)
nullseries = df.isnull().sum()
nullseries[nullseries > 0]

## Calculate LTV

To calculate LTV, referred to https://blog.hubspot.com/service/how-to-calculate-customer-lifetime-value. 

Define the variables as follows:

LTV = (Avg Income Generated by Driver/Churn Rate) * 365

Avg Income Generated by Driver = Total Income Generated by Driver /Number of Days Driver Drove

Churn Rate = (Number of Drivers Who Have Stopped Driving/Total Number of Drivers) * 100%

In [ ]:
# Churn Rate
churn_rate = len(df[df['churn'] == 1]) / len(df)
churn_rate

## What is the average projected lifetime of a driver? 

That is, once a driver is onboarded, how long do they typically continue driving with Lyft?

In [ ]:
### Average Lifetime of a driver is inverse of churn rate.
print('Average lifetime of driver: ', 1/churn_rate, 'years.')

# Recommended Driver's Lifetime Value 

value of a driver to Lyft over the entire projected lifetime of a driver

## Equation: 

Assumptions: 3 months of data is enough to define LTV.

In [ ]:
df['average_ride_val'] = df['avg_fare'] / df['ride_count']

df['average_ride_freq'] = df['ride_count'] / df['unique_drive_days']

df['average_val'] = df['average_ride_val'] * df['average_ride_freq']

df['ltv'] = df['average_val'] * 365 / churn_rate

## Churn Feature Importance

In [ ]:
# Using Pycaret to see what may contribute to churn

from pycaret.classification import *
s = setup(df, target = 'churn', ignore_features = ['driver_id'])

In [ ]:
# compare all models
best_model = compare_models(sort='AUC')

In [ ]:
# print best_model parameters
print(best_model)

In [ ]:
# tune best model
tuned_best_model = tune_model(best_model)

In [ ]:
# AUC Plot
plot_model(tuned_best_model, plot='auc')

In [ ]:
# Feature Importance Plot
plot_model(tuned_best_model, plot = 'feature')

Driving tenure is the best indicator for churn. How long a driver has been on the platform/how long they have driven for is indicator for churning. Meaning those who are new to Lyft are the most prone to churning.

## Segment the driver population to identify driving behavior that may lead to churn

In [ ]:
plt.figure(figsize=(8,8))
sns.boxplot(x="churn", y="driving_period", data=df)
plt.show()

There is a difference.

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,8))
sns.distplot(df[df['churn']== 0]['avg_next_ride_lag'],ax=ax[0])
ax[0].set_title('Average Time to Accept Next Ride in Non-Churn')
sns.distplot(df[df['churn']== 1]['avg_next_ride_lag'],ax=ax[1])
ax[1].set_title('Average Time to Accept Next Ride in Churn')
plt.show()

There is a slight broader distribution with non-churners who end up picking up another ride.

In [ ]:
a = sns.FacetGrid(df, hue = 'churn', aspect=4 )
a.map(sns.kdeplot, 'unique_drive_days', shade= True )
a.set(xlim=(df['unique_drive_days'].min() , df['unique_drive_days'].max()))
a.add_legend()

We can see here that as Feature Importance suggested, distinct number of days driver has driven leads to churn. We see here that those who churned only drove for 10 days. This directly relates to the biggest contributor to churn: driver's driving period/tenure.

In [ ]:
plt.figure(figsize=[16,12])

plt.subplot(231)
sns.boxplot(x="churn", y="avg_accept_dur_sec", data=df)
plt.title('Time to Next Ride (within same date)')
plt.xlabel('Churn')
plt.ylabel('Average Time to Accept Next Ride')

plt.subplot(232)
sns.boxplot(x="churn", y="ride_count", data=df)
plt.title('Total Rides')
plt.xlabel('Churn')
plt.ylabel('Total Rides')

plt.subplot(233)
sns.violinplot(x="churn", y="avg_fare", data=df)
plt.title('Average Fare')
plt.xlabel('Churn')
plt.ylabel('Average Fare')

Biggest difference between churn and non-churn between these three is with total rides which makes sense. Those who drop off Lyft will have overall least amount of rides in 3 month period.

## LTV Feature Importance

In [ ]:
# Using Pycaret to see what may contribute to LTV

from pycaret.regression import *
s2 = setup(df, target = 'ltv', ignore_features = ['driver_id', 'average_val'])

In [ ]:
# compare all models
best_model2 = compare_models(sort='RMSE')

In [ ]:
# print best_model parameters
print(best_model2)

In [ ]:
# tune best model
tuned_best_model2 = tune_model(best_model2) 

In [ ]:
# RMSE Plot
plot_model(tuned_best_model2, plot = 'residuals')

In [ ]:
# Feature Importance Plot
plot_model(tuned_best_model2, plot = 'feature')

## Main factors that affect a driver's lifetime value are: 
* number of days driver has driven for
* ride cost
* usual fare for each driver's ride
* how long in distance each ride is
* how long in duration each ride is

In [ ]:
df.plot.scatter(x='unique_drive_days', y='ltv')

This is interesting. (Negative correlation). This shows that the least unique drive days have highest ltv. What this is showing that those who bring greatest value to Lyft drive less (have high volume of rides per day). 

In [ ]:
df.plot.scatter(x='average_ride_val',y='ltv')

Average cost per ride does not really show much correlation with ltv.

In [ ]:
df.plot.scatter(x='avg_fare',y='ltv')

In [ ]:
df.plot.scatter(x='avg_ride_distance',y='ltv')

Last two do not have much correlation. Hence, they were low ranked in feature importance.

## Business Recommendations

Detailed in Google Slides Presentation.